⚠️ ⚠️ ⚠️  Below code is not right an attempt to user autgen agents to work with webscraping package ⚠️ 

In [ ]:
pip install python-dotenv

In [ ]:
!pip install autogen-agentchat autogen-ext openai tiktoken langgraph autogen

In [ ]:
# from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json, register_function
import autogen
import os

In [ ]:
from dotenv import load_dotenv

load_dotenv() 

model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

In [ ]:
import nest_asyncio

# Apply nest_asyncio to resolve event loop issues
nest_asyncio.apply()

In [ ]:
# Replace with your actual OpenAI API key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")



In [ ]:
import json
from scrapegraphai.graphs import SmartScraperMultiGraph

In [ ]:
prompt="""
  Extract current job market insights for tech roles:
  - Average salaries for software engineers
  - Most in-demand programming languages
  - Top tech companies hiring
  - Emerging job trends in tech
  """

In [50]:
def scrape_pages(urls: list, prompt: str, model: str = "gpt-4o-mini") -> dict:
    """
    Scrape multiple URLs using SmartScraperMultiGraph from scrapegraphai.
    
    Args:
        urls (list): List of URLs to scrape.
        prompt (str): The scraping prompt (e.g., "Extract title and main content").
        model (str): LLM model to use (default: gpt-4o-mini).
    
    Returns:
        dict: Scraped data for each URL.
    """
    # Define configuration for ScrapegraphAI
    graph_config_openai = {
        "llm": {
            "api_key": OPENAI_API_KEY,
            "model": "openai/gpt-4o-mini",
        },
        "verbose": True,
        "headless": False,

    }
        
    # Scrape job market information from multiple sources
    multiple_job_graph = SmartScraperMultiGraph(
        prompt=prompt,
        source=[
            "https://www.levels.fyi/",
            "https://stackoverflow.com/jobs/companies",
            "https://www.indeed.com/career/software-engineer"
        ],
        config=graph_config_openai
    )

    # define a schema for structured output:
    schema = {
        "type": "object",
        "properties": {
            "headlines": {"type": "array", "items": {"type": "string"}}
        }
    }

    multiple_job_graph.schema = schema
    
    # Run the scraper
    result = multiple_job_graph.run()
    
    # Return the scraped data as a dictionary
    return result



In [ ]:
# AutoGen setup
# Load configuration (e.g., for OpenAI API)
config_list = config_list_from_json(
    "OAI_CONFIG_LIST.json",  # Ensure this file exists with your API key
    filter_dict={"model": ["gpt-4o-mini"]},
)

# Initialize agents
assistant = AssistantAgent(
    name="ScraperAssistant",
    llm_config={"config_list": config_list},
    system_message="I can scrape web pages using SmartScraperMultiGraph. Provide URLs and a prompt to extract data."
)

In [ ]:
user_proxy = UserProxyAgent(
    name="UserProxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
    code_execution_config={"work_dir": "coding"},
)


In [ ]:
register_function(
    scrape_pages,
    caller=assistant,
    executor=user_proxy,
    name="scrape_page",
    description="Scrape a web page for job listings and return content."
)

In [ ]:
user_proxy.initiate_chat(
    assistant,
    message="Scrape these URLs with the prompt 'Extract jobs information from site"
)